### Using Rbdyn and SpaceVecAlg
1. The library comes with python bindings using cython transcription. Thus, fast prototyping can be done to verify algorithms using the provide python functions.

2. To setup the libraries, we will need to install some common components first.
<br>`sudo apt install apt-transport-https lsb-release`

3. *Eigen3ToPython*
<br>1) `sudo apt-key adv --keyserver 'hkp://keyserver.ubuntu.com:80' --recv-key 892EA6EE273707C6495A6FB6220D644C64666806`
<br>2) `sudo sh -c 'echo "deb https://dl.bintray.com/gergondet/multi-contact-release $(lsb_release -sc) main" | sudo tee -a /etc/apt/sources.list.d/multi-contact.list'`
<br>3) `sudo apt update && sudo apt install python-eigen python3-eigen`

4. *SpaceVecAlg*
<br>1) `sudo apt update && sudo apt install libspacevecalg-dev python-spacevecalg python3-spacevecalg`

5. *RBdyn*
<br>1) `sudo apt update && sudo apt install librbdyn-dev python-rbdyn python3-rbdyn`

In [ ]:
import pybullet as b
import time
import pybullet_data
import eigen as e
import rbdyn as rbd
import sva as s
import numpy as np
from rbdyn.parsers import *

DT = 0.002 #s
MOVEMENT_TIME = 10000 #ms
URDF = "./resource/kuka_iiwa.urdf"

def get_rot(T):
    R = e.Matrix3d()
    for row in range(R.rows()):
        for col in range(R.cols()):
            R.coeff(row, col, T.coeff(row, col))
    return R

def get_pos(T):
    p = e.Vector3d()
    for i in range(p.rows()):
        p[i] = T.coeff(i,3)
    return p

def get_col(R, c=0):
    assert c <= R.cols(), "column index out of bounds"
    assert R.rows() == R.cols(), "square matrix expected"
    assert R.rows() == 3, "3x3 matrix expected"
    v = e.Vector3d()
    for i in range(R.rows()):
        v[i] = R.coeff(i,c)
    return v

def get_orientation_error(R, R_d):
    '''Calculates orientation error
    @param R Current orientation in Mat3d
    @param R_d Desired orientation in Mat3d
    @return del_phi "R_d-R" error
    '''
    c1 = get_col(R,0)
    c2 = get_col(R,1)
    c3 = get_col(R,2)
    d1 = get_col(R_d,0)
    d2 = get_col(R_d,1)
    d3 = get_col(R_d,2)
    
    return 1.0/2.0*(c1.cross(d1)+c2.cross(d2)+c3.cross(d3))
    
    
def body_id_from_name(name, bodies):
    '''Gets the body Id from the body name
    @param name The name of the body
    @param bodies The set of bodies provided by the multibody data structure
    @return Id of the body, -1 if not found
    '''
    for bi, b in enumerate(bodies):
        if (b.name().decode("utf-8") == name):
            return bi
    return -1

def sva_to_affine(s):
    '''Converts a spatial transform matrix to a homogeneous transform matrix
    @param s Spatial transform
    @return Homogeneous transform matrix
    '''
    m4d = e.Matrix4d.Identity()
    R = s.rotation().transpose()
    p = s.translation();
    
    for row in range(R.rows()):
        for col in range(R.cols()):
            m4d.coeff(row, col, R.coeff(row, col))
    for row in range(p.rows()):
        m4d.coeff(row, 3, p[row]);
        
    return m4d

def inv_spd_sr(mat, k=1e-4):
    '''Converts an inverse into an SVD problem with singularity robust inverse
    @param mat Matrix to be inversed
    @param tolerance Tolerance to the minimum singular value that can go through reciprocal operation
    @return Homogeneous transform matrix
    '''
    mat_np = np.array(mat)
    reg = k * np.identity(np.size(mat_np,0))
    
    return e.MatrixXd(mat_np.transpose().dot(np.linalg.inv(mat_np.dot(mat_np.transpose()) + reg)))
    
# import robot as global variable
p = from_urdf_file(URDF)
print("Imported " + p.name.decode("utf-8"))
# set gravity direction (this is the acceleration at base joint for RNEA)
p.mbc.gravity = e.Vector3d(0,0,9.81)
p.mbc.zero(p.mb)

# setup physics
physicsClient = b.connect(b.GUI)
b.setAdditionalSearchPath(pybullet_data.getDataPath()) #used by loadURDF
b.setGravity(0,0,-9.81)
b.setRealTimeSimulation(0)
b.setTimeStep(DT)

# import robot
planeId = b.loadURDF("plane.urdf")
startPos = [0,0,0]
startOrientation = b.getQuaternionFromEuler([0,0,0])
loadFlag = b.URDF_USE_INERTIA_FROM_FILE | b.URDF_USE_SELF_COLLISION_EXCLUDE_ALL_PARENTS
robotId = b.loadURDF(URDF,startPos, startOrientation, flags=loadFlag)
nDof = b.getNumJoints(robotId)
jointsId = range(nDof)

# gains
Kp = e.MatrixXd(np.diag([400,400,400,900,900,900]))
Kd = e.MatrixXd(np.diag([40,40,40,60,60,60]))

# selection matrix to swap linear and angular components
S = e.MatrixXd(6,6)
S.coeff(0,3,1)
S.coeff(1,4,1)
S.coeff(2,5,1) # linear
S.coeff(3,0,1)
S.coeff(4,1,1)
S.coeff(5,2,1) # angular

# task definitions
bodyName1 = "lbr_iiwa_link_7"
S1 = S

bodyName2 = "lbr_iiwa_link_3"
S2 = e.MatrixXd(3,6)
S2.coeff(0,3,1)
S2.coeff(1,4,1)
S2.coeff(2,5,1)

# setup configuration
b.setJointMotorControlArray(robotId, jointsId, b.VELOCITY_CONTROL, forces=[0] * nDof)
q_home = np.zeros(nDof)#0.1*np.arange(nDof)
for i in jointsId:
    b.resetJointState(robotId, i, targetValue=q_home[i])
q_home = e.VectorXd(q_home)
    
# simulate torque control
for i in range(1000000):
    joint_states = b.getJointStates(robotId, jointsId)
    
    # update q and dq states from simulation
    q = [joint_states[i][0] for i in jointsId]
    dq = [joint_states[i][1] for i in jointsId]
    for i in range(0, nDof):
        p.mbc.q[i+1][0] = q[i]
        p.mbc.alpha[i+1][0] = dq[i]

    # forward kinematics
    rbd.forwardKinematics(p.mb, p.mbc)
    rbd.forwardVelocity(p.mb, p.mbc)
    
    # convert
    q = e.VectorXd(q)
    dq = e.VectorXd(dq)
    
    # pose
    ofsRot = e.Matrix3d.Identity()
    ofsPos = e.Vector3d(0,0,0)
    T1 = sva_to_affine(s.PTransformd(ofsRot.transpose(), ofsPos) * p.mbc.bodyPosW[body_id_from_name(bodyName1, p.mb.bodies())])
    T2 = sva_to_affine(s.PTransformd(ofsRot.transpose(), ofsPos) * p.mbc.bodyPosW[body_id_from_name(bodyName2, p.mb.bodies())])
    
    # jacobian, dJacobian [angular; linear]
    jac = rbd.Jacobian(p.mb, bodyName1.encode('utf-8'), ofsPos)
    tempJ1 = e.MatrixXd(6, p.mb.nrDof());
    tempdJ1 = e.MatrixXd(6, p.mb.nrDof());
    jac.fullJacobian(p.mb, jac.jacobian(p.mb, p.mbc), tempJ1);
    jac.fullJacobian(p.mb, jac.jacobianDot(p.mb, p.mbc), tempdJ1);
    J1 = S*tempJ1
    dJ1 = S*tempdJ1
    
    # jacobian, dJacobian [angular; linear]
    jac = rbd.Jacobian(p.mb, bodyName2.encode('utf-8'), ofsPos)
    tempJ2 = e.MatrixXd(6, p.mb.nrDof());
    tempdJ2 = e.MatrixXd(6, p.mb.nrDof());
    jac.fullJacobian(p.mb, jac.jacobian(p.mb, p.mbc), tempJ2);
    jac.fullJacobian(p.mb, jac.jacobianDot(p.mb, p.mbc), tempdJ2);
    J2 = S2*tempJ2
    dJ2 = S2*tempdJ2
    
    # mass matrix
    fd = rbd.ForwardDynamics(p.mb)
    fd.computeH(p.mb, p.mbc);
    M = fd.H()

    # nonlinear effects
    fd = rbd.ForwardDynamics(p.mb)
    fd.computeC(p.mb, p.mbc);
    h = fd.C()
    
    # compute Minv
    Minv = M.inverse()
    
    # compute lambda1, Jbar1, Nprec1
    lambda1 = inv_spd_sr(J1*Minv*J1.transpose(),1e-3)
    J1bar = Minv*J1.transpose()*lambda1
    Nprec1 = e.MatrixXd.Identity(nDof, nDof) - J1bar*J1
    
    # compute lambda21, Jbar21
    J21 = J2*Nprec1
    lambda21 = inv_spd_sr(J21*Minv*J21.transpose(),1e-3)
    J21bar = Minv*J21.transpose()*lambda21
    Nprec2 = Nprec1 * (e.MatrixXd.Identity(nDof, nDof) - J21bar*J21)
    
    # error
    err1 = e.VectorXd(S.rows())
    x1_des = e.Vector3d(0.5,0,0.5)
    R1_des = e.MatrixXd(3,3)
    R1_des.coeff(1,0,1)
    R1_des.coeff(2,1,1)
    R1_des.coeff(0,2,1)
    err1_pos = x1_des - get_pos(T1)
    err1_rot = get_orientation_error(get_rot(T1), R1_des)
    for i in range(err1_pos.rows()):
        err1[i] = err1_pos[i]
        err1[i+err1_pos.rows()] = err1_rot[i]
    
    derr1 = -J1*dq
    err2 = e.VectorXd(S.rows())
    err2[0] = T2.coeff(0,3)
    err2[1] = T2.coeff(1,3)
    err2[2] = T2.coeff(2,3)
    derr2 = -J2*dq
    
    # control
    tau_1 = e.VectorXd(J1.transpose()*lambda1*(Kp*err1+Kd*derr1-dJ1*dq))
    tau_2 = e.VectorXd(J21.transpose()*lambda21*(400*err2+40*derr2-dJ2*dq-J2*Minv*tau_1))
    tau = tau_1 + Nprec1.transpose()*tau_2 + h

    # command system:
    for i in jointsId:
        b.setJointMotorControl2(robotId, i, b.TORQUE_CONTROL, force=tau[i])
    
    b.stepSimulation()
    time.sleep(DT)
b.disconnect()

Imported lbr_iiwa


In [ ]:
import pybullet as b
import time
import pybullet_data
import eigen as e
import rbdyn as rbd
import sva as s
import numpy as np
import math
from rbdyn.parsers import *

DT = 0.002 #s
MOVEMENT_TIME = 10000 #ms
URDF = "./resource/kuka_iiwa_dual.urdf"

def get_rot(T):
    R = e.Matrix3d()
    for row in range(R.rows()):
        for col in range(R.cols()):
            R.coeff(row, col, T.coeff(row, col))
    return R

def get_pos(T):
    p = e.Vector3d()
    for i in range(p.rows()):
        p[i] = T.coeff(i,3)
    return p

def get_col(R, c=0):
    assert c <= R.cols(), "column index out of bounds"
    assert R.rows() == R.cols(), "square matrix expected"
    assert R.rows() == 3, "3x3 matrix expected"
    v = e.Vector3d()
    for i in range(R.rows()):
        v[i] = R.coeff(i,c)
    return v

def get_orientation_error(R, R_d):
    '''Calculates orientation error
    @param R Current orientation in Mat3d
    @param R_d Desired orientation in Mat3d
    @return del_phi "R_d-R" error
    '''
    c1 = get_col(R,0)
    c2 = get_col(R,1)
    c3 = get_col(R,2)
    d1 = get_col(R_d,0)
    d2 = get_col(R_d,1)
    d3 = get_col(R_d,2)
    
    return 1.0/2.0*(c1.cross(d1)+c2.cross(d2)+c3.cross(d3))

def body_id_from_name(name, bodies):
    '''Gets the body Id from the body name
    @param name The name of the body
    @param bodies The set of bodies provided by the multibody data structure
    @return Id of the body, -1 if not found
    '''
    for bi, b in enumerate(bodies):
        if (b.name().decode("utf-8") == name):
            return bi
    return -1

def sva_to_affine(s):
    '''Converts a spatial transform matrix to a homogeneous transform matrix
    @param s Spatial transform
    @return Homogeneous transform matrix
    '''
    m4d = e.Matrix4d.Identity()
    R = s.rotation().transpose()
    p = s.translation();
    
    for row in range(3):
        for col in range(3):
            m4d.coeff(row, col, R.coeff(row, col))
    for row in range(3):
        m4d.coeff(row, 3, p[row]);
        
    return m4d

def inv_spd_sr(mat, k=1e-4):
    '''Converts an inverse into an SVD problem with singularity robust inverse
    @param mat Matrix to be inversed
    @param tolerance Tolerance to the minimum singular value that can go through reciprocal operation
    @return Homogeneous transform matrix
    '''
    mat_np = np.array(mat)
    reg = k * np.identity(np.size(mat_np,0))
    
    return e.MatrixXd(mat_np.transpose().dot(np.linalg.inv(mat_np.dot(mat_np.transpose()) + reg)))

# import robot as global variable
p = from_urdf_file(URDF)
print("Imported " + p.name.decode("utf-8"))
# set gravity direction (this is the acceleration at base joint for RNEA)
p.mbc.gravity = e.Vector3d(0,0,9.81)
p.mbc.zero(p.mb)

# setup physics
physicsClient = b.connect(b.GUI)
b.setAdditionalSearchPath(pybullet_data.getDataPath()) #used by loadURDF
b.setGravity(0,0,-9.81)
b.setRealTimeSimulation(0)
b.setTimeStep(DT)

# import robot
planeId = b.loadURDF("plane.urdf")
startPos = [0,0,0]
startOrientation = b.getQuaternionFromEuler([0,0,0])
loadFlag = b.URDF_USE_INERTIA_FROM_FILE | b.URDF_USE_SELF_COLLISION_EXCLUDE_ALL_PARENTS
robotId = b.loadURDF(URDF,startPos, startOrientation, flags=loadFlag)
nDof = b.getNumJoints(robotId)
jointsId = range(nDof)

# gains
Kp = e.MatrixXd(np.diag([1600,1600,1600,3600,3600,3600]))
Kd = e.MatrixXd(np.diag([40,40,40,40,40,40]))

# selection matrix to swap linear and angular components
S = e.MatrixXd(6,6)
S.coeff(0,3,1)
S.coeff(1,4,1)
S.coeff(2,5,1)
S.coeff(3,0,1)
S.coeff(4,1,1)
S.coeff(5,2,1)

# task definitions
bodyName1 = "lbr_left_7"
S1 = S

bodyName2 = "lbr_right_7"
S2 = S

bodyName3 = "lbr_left_3"
S3 = e.MatrixXd(3,6)
S3.coeff(0,3,1)
S3.coeff(1,4,1)
S3.coeff(2,5,1)

bodyName4 = "lbr_right_3"
S4 = e.MatrixXd(3,6)
S4.coeff(0,3,1)
S4.coeff(1,4,1)
S4.coeff(2,5,1)

# setup configuration
b.setJointMotorControlArray(robotId, jointsId, b.VELOCITY_CONTROL, forces=[0] * nDof)
# q_home = np.zeros(nDof)#0.1*np.arange(nDof)
q_home = [0.761946,0.702505,-0.257926,-1.45529,1.16864,0.874719,-0.146405,2.75053,-0.708697,-0.309352,1.45537,-0.745851,-0.66836,1.50138]
for i in jointsId:
    b.resetJointState(robotId, i, targetValue=q_home[i])
q_home = e.VectorXd(q_home)

# simulate torque control
for k in range(10000000):
    joint_states = b.getJointStates(robotId, jointsId)
    
    # update q and dq states from simulation
    q = [joint_states[i][0] for i in jointsId]
    dq = [joint_states[i][1] for i in jointsId]
    for i in range(0, nDof):
        p.mbc.q[i+1][0] = q[i]
        p.mbc.alpha[i+1][0] = dq[i]

    # forward kinematics
    rbd.forwardKinematics(p.mb, p.mbc)
    rbd.forwardVelocity(p.mb, p.mbc)
    
    # convert
    q = e.VectorXd(q)
    dq = e.VectorXd(dq)
    
    # pose
    ofsRot = e.Matrix3d.Identity()
    ofsPos = e.Vector3d(0,0,0)
    T1 = sva_to_affine(s.PTransformd(ofsRot.transpose(), ofsPos) * p.mbc.bodyPosW[body_id_from_name(bodyName1, p.mb.bodies())])
    T2 = sva_to_affine(s.PTransformd(ofsRot.transpose(), ofsPos) * p.mbc.bodyPosW[body_id_from_name(bodyName2, p.mb.bodies())])
#     T3 = sva_to_affine(s.PTransformd(ofsRot.transpose(), ofsPos) * p.mbc.bodyPosW[body_id_from_name(bodyName3, p.mb.bodies())])
#     T4 = sva_to_affine(s.PTransformd(ofsRot.transpose(), ofsPos) * p.mbc.bodyPosW[body_id_from_name(bodyName4, p.mb.bodies())])
    
    # jacobian, dJacobian [angular; linear]
    jac = rbd.Jacobian(p.mb, bodyName1.encode('utf-8'), ofsPos)
    tempJ1 = e.MatrixXd(6, p.mb.nrDof());
    tempdJ1 = e.MatrixXd(6, p.mb.nrDof());
    jac.fullJacobian(p.mb, jac.jacobian(p.mb, p.mbc), tempJ1);
    jac.fullJacobian(p.mb, jac.jacobianDot(p.mb, p.mbc), tempdJ1);
    J1 = S1*tempJ1
    dJ1 = S1*tempdJ1
    
    # jacobian, dJacobian [angular; linear]
    jac = rbd.Jacobian(p.mb, bodyName2.encode('utf-8'), ofsPos)
    tempJ2 = e.MatrixXd(6, p.mb.nrDof());
    tempdJ2 = e.MatrixXd(6, p.mb.nrDof());
    jac.fullJacobian(p.mb, jac.jacobian(p.mb, p.mbc), tempJ2);
    jac.fullJacobian(p.mb, jac.jacobianDot(p.mb, p.mbc), tempdJ2);
    J2 = S2*tempJ2
    dJ2 = S2*tempdJ2
    
#     # jacobian, dJacobian [angular; linear]
#     jac = rbd.Jacobian(p.mb, bodyName1.encode('utf-8'), ofsPos)
#     tempJ3 = e.MatrixXd(6, p.mb.nrDof());
#     tempdJ3 = e.MatrixXd(6, p.mb.nrDof());
#     jac.fullJacobian(p.mb, jac.jacobian(p.mb, p.mbc), tempJ3);
#     jac.fullJacobian(p.mb, jac.jacobianDot(p.mb, p.mbc), tempdJ3);
#     J3 = S3*tempJ3
#     dJ3 = S3*tempdJ3
    
#     # jacobian, dJacobian [angular; linear]
#     jac = rbd.Jacobian(p.mb, bodyName2.encode('utf-8'), ofsPos)
#     tempJ4 = e.MatrixXd(6, p.mb.nrDof());
#     tempdJ4 = e.MatrixXd(6, p.mb.nrDof());
#     jac.fullJacobian(p.mb, jac.jacobian(p.mb, p.mbc), tempJ4);
#     jac.fullJacobian(p.mb, jac.jacobianDot(p.mb, p.mbc), tempdJ4);
#     J4 = S4*tempJ4
#     dJ4 = S4*tempdJ4
    
    # mass matrix
    fd = rbd.ForwardDynamics(p.mb)
    fd.computeH(p.mb, p.mbc);
    M = fd.H()

    # nonlinear effects
    fd = rbd.ForwardDynamics(p.mb)
    fd.computeC(p.mb, p.mbc);
    h = fd.C()
    
    # compute Minv
    Minv = M.inverse()
    
    # compute lambda1, Jbar1, Nprec1
    lambda1 = inv_spd_sr(J1*Minv*J1.transpose(),1e-3)
    J1bar = Minv*J1.transpose()*lambda1
    Nprec1 = e.MatrixXd.Identity(nDof, nDof) - J1bar*J1

    # compute lambda21, Jbar21
    J21 = J2*Nprec1
    lambda21 = inv_spd_sr(J21*Minv*J21.transpose(),1e-3)
    J21bar = Minv*J21.transpose()*lambda21
    Nprec2 = Nprec1 * (e.MatrixXd.Identity(nDof, nDof) - J21bar*J21)
    
#     # compute lambda321, Jbar321
#     J21 = J2*Nprec2
#     lambda21 = inv_spd_sr(J21*Minv*J21.transpose(),1e-3)
#     J21bar = Minv*J21.transpose()*lambda21
#     Nprec2 = Nprec1 * (e.MatrixXd.Identity(nDof, nDof) - J21bar*J21)
    
    # error
    err1 = e.VectorXd(S.rows())
    x1_des = e.Vector3d(0.5+0.1*math.sin(2*np.pi*1*k*DT),0.2+0.1*math.cos(2*np.pi*1*k*DT),0.7)
    R1_des = e.MatrixXd(3,3)
    R1_des.coeff(0,0,-1)
    R1_des.coeff(1,2,-1)
    R1_des.coeff(2,1,-1)
    err1_pos = x1_des - get_pos(T1)
    err1_rot = get_orientation_error(get_rot(T1), R1_des)
    for i in range(err1_pos.rows()):
        err1[i] = err1_pos[i]
        err1[i+err1_pos.rows()] = err1_rot[i]
    derr1 = -J1*dq
    
    err2 = e.VectorXd(S.rows())
    x2_des = e.Vector3d(0.5+0.1*math.sin(2*np.pi*1*k*DT),-0.2+0.1*math.cos(2*np.pi*1*k*DT),0.7)
    R2_des = e.MatrixXd(3,3)
    R2_des.coeff(0,1,-1)
    R2_des.coeff(1,2,1)
    R2_des.coeff(2,0,-1)
    err2_pos = x2_des - get_pos(T2)
    err2_rot = get_orientation_error(get_rot(T2), R2_des)
    for i in range(err2_pos.rows()):
        err2[i] = err2_pos[i]
        err2[i+err2_pos.rows()] = err2_rot[i]
    derr2 = -J2*dq
    
    err3 = e.VectorXd(q_home) - q;
    derr3 = -dq
    
    # control
    tau_1 = e.VectorXd(J1.transpose()*lambda1*(Kp*err1+Kd*derr1-dJ1*dq))
    tau_2 = e.VectorXd(J21.transpose()*lambda21*(Kp*err2+Kd*derr2-dJ2*dq-J2*Minv*tau_1))
    tau_3 = 400*err3+40*derr3
    tau = tau_1 + Nprec1.transpose()*tau_2 + Nprec2.transpose()*tau_3 + h

    # command system:
    for i in jointsId:
        b.setJointMotorControl2(robotId, i, b.TORQUE_CONTROL, force=tau[i])
    
    b.stepSimulation()
    time.sleep(DT)
b.disconnect()

Imported lbr_iiwa
